In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from typing import TypeVar, Any
from pydantic import BaseModel, Field, create_model

import litellm
from litellm import completion
from instructor import from_litellm, Mode



In [10]:
litellm.drop_params = True  # watsonx.ai doesn't support `json_mode`
client = from_litellm(completion, mode=Mode.JSON)  # create an instructor client from litellm

In [ ]:
class BaseResponse(BaseModel):
    """A default response model that stores a list of predicted Ekman emotions. We will use this to predict the emotions of a review."""
    answer: str

ResponseType = TypeVar('ResponseType', bound=BaseModel)

class LLMCaller:
    """
    A class to interact with a Large Language Model (LLM)
    using the LiteLLM and Instructor libraries.
    
    Designed to send prompts and receive structured responses
    as Pydantic models (e.g., predicted emotions).
    """
    def __init__(self, api_key: str, project_id: str, api_url: str, model_id: str, params: dict[str, Any]):
        """Initializes the LLMCaller with Watsonx credentials and configuration."""
        self.api_key = api_key
        self.project_id = project_id
        self.api_url = api_url
        self.model_id = model_id
        self.params = params

        litellm.drop_params = True
        self.client = from_litellm(completion, mode=Mode.JSON)

    def create_response_model(self, title: str, fields: dict) -> ResponseType:
        return create_model(title, **fields, __base__=BaseResponse)

    def invoke(self, prompt: str, response_model: ResponseType = BaseResponse, **kwargs) -> ResponseType:
        response = self.client.chat.completions.create(
            model=self.model_id,
            messages=[{
                "role": "user",
                "content": prompt + "\n\nRespond using this structure: " + str(response_model.__annotations__)
            }],
            project_id=self.project_id,
            apikey=self.api_key,
            api_base=self.api_url,
            response_model=response_model,
            **kwargs
        )
        return response





In [3]:
load_dotenv()

llm = LLMCaller(
    api_key=os.getenv("WX_API_KEY"),
    project_id=os.getenv("WX_PROJECT_ID_RAG"),
    api_url=os.getenv("WX_URL"),
    model_id="watsonx/mistralai/mistral-large",
    params={"max_tokens": 100}
)


In [4]:
# Load the emotion definitions
df_defs = pd.read_csv("data/oxford_ekman_emotions.csv")
emotion_definitions = dict(zip(df_defs["emotion"], df_defs["definition"]))



In [5]:
class EmotionResponse(BaseModel):
    emotions: list[str] = Field(..., description="The list of Ekman emotions expressed in the review.")


In [6]:
def build_prompt(review: str, definitions: dict[str, str]) -> str:
    intro = "You are a helpful assistant that classifies customer reviews using Ekman's 7 emotions.\n"
    guide = "Use ONLY the following emotion definitions:\n\n"
    for emotion, definition in definitions.items():
        guide += f"{emotion.upper()}: {definition}\n"
    task = (
        f"\n\nClassify the following review into one or more Ekman emotions:\n\n"
        f"Review: \"{review}\"\n"
        f"List all matching emotions. If none apply, return ['neutral']."
    )
    return intro + guide + task


In [7]:
sample_review = "I was furious at the lack of help and felt completely disrespected."

prompt = build_prompt(sample_review, emotion_definitions)
result = llm.invoke(prompt=prompt, response_model=EmotionResponse)

print("Prompt sent to LLM:")
print(prompt)

try:
    result = llm.invoke(prompt=prompt, response_model=EmotionResponse)
    print("Predicted emotions:", result.emotions)
except Exception as e:
    print("Error occurred:", e)


Prompt sent to LLM:
You are a helpful assistant that classifies customer reviews using Ekman's 7 emotions.
Use ONLY the following emotion definitions:

ANGER: A strong feeling of annoyance, displeasure, or hostility.
DISGUST: A strong feeling of dislike or disapproval for something unpleasant or offensive.
FEAR: An unpleasant emotion caused by the threat of danger, pain, or harm.
JOY: A feeling of great pleasure and happiness.
SADNESS: The condition or quality of being sad; sorrow; a feeling of unhappiness or grief.
SURPRISE: A feeling of mild astonishment or shock caused by something unexpected.
NEUTRAL: Not displaying any strong emotion or feeling; a lack of emotional expression.


Classify the following review into one or more Ekman emotions:

Review: "I was furious at the lack of help and felt completely disrespected."
List all matching emotions. If none apply, return ['neutral'].
Predicted emotions: ['ANGER', 'DISGUST']


In [8]:
df_test = pd.read_csv("data/ekman_test.csv").sample(n=500, random_state=42)

def safe_predict_emotions(text):
    try:
        prompt = build_prompt(text, emotion_definitions)
        result = llm.invoke(prompt=prompt, response_model=EmotionResponse)
        return result.emotions
    except Exception as e:
        return ["error"]

df_test["predicted_emotions"] = df_test["text"].apply(safe_predict_emotions)


In [9]:
df_test.to_csv("ekman_test_predictions_structured.csv", index=False)

In [176]:
import ast

def classify_emotion(review_text):
    # Build the prompt using correct variables
    prompt = build_prompt(review_text, emotion_definitions, few_shots)
    
    # Get the model's response
    response = llm.invoke(prompt).strip()

    # Try parsing it as a Python list
    try:
        parsed = ast.literal_eval(response)
        if isinstance(parsed, list):
            return parsed
        else:
            return f"Parsed but not a list: {parsed}"
    except Exception as e:
        return f"Could not parse response. Raw output: {response}"


In [177]:
user_review = "This game crashes constantly and the support team never responds!"
prompt = build_prompt(user_review, emotion_definitions, few_shots)
print(prompt)

You are an assistant that identifies all applicable emotions from a customer review.

Emotion Definitions:


Examples:
Input: "I felt abandoned and hopeless."
Emotions: sadness

Input: "That jump scare was wild!"
Emotions: surprise, fear

Now classify the following text. Return your answer only as a Python list of one or more emotion labels from this set:
["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"].

Do not explain. Do not leave the Emotions empty!.
Input: "This game crashes constantly and the support team never responds!"
Emotions:
